# DSE Course 2, Session 7: Casting Defect Detection Case Study

**Instructor**: Wesley Beckner

**Contact**: wesleybeckner@gmail.com

<br>

---

<br>

# Overview

This dataset contains data from machining experiments. Machining data was collected from a CNC machine for variations of tool condition, feed rate, and clamping pressure.

## Definitions

* feed rate
    * relative velocity of the cutting tool along the workpiece (mm/s)
* clamping pressure
    * pressure used to hold the workpiece in the vise (bar)

In 18 machining experiments, time series data was collected with a sampling rate of 100 ms from the 4 motors in the CNC (X, Y, Z axes and spindle).
And output of each experiments includes tool condition (unworn and worn tools) and whether or not the tool passed visual inspection.
## Objectives

* tool wear detection 
* detection of inadequate clamping.

[more info](https://github.com/wesleybeckner/ds_for_engineers/tree/main/data/mill_tool_wear)

<br>

---

<br>

<a name='top'></a>

# Contents

* 7.0 [Preparing Environment and Importing Data](#x.0)
  * 7.0.1 [Import Packages](#x.0.1)
  * 7.0.2 [Load Dataset](#x.0.2)
* 7.1 [Exploratory Data Analysis](#x.1)
  * 7.1.1 [First Look: Shape, Nulls, Description](#x.1.1)
  * 7.1.2 [Descriptive Statistics](#x.1.2)
    * 7.1.2.1 [Univariate Analysis](#x.1.2.1)
    * 7.1.2.2 [Multivariate Analysis](#x.1.2.2)
    * 7.1.2.3 [Frequency Analysis](#x.1.2.3)
* 7.2 [Feature Engineering](#x.2)
  * 7.2.1 [Feature Skewness](#x.2.1)
  * 7.2.2 [Feature Colinearity](#x.2.2)
  * 7.2.3 [Feature Normalization](#x.2.3)
  * 7.2.4 [Feature Selection](#x.2.4)
  * 7.2.5 [Fast Fourier Transform](#x.2.5)
* 7.3 [Modeling](#x.3)
  * 7.3.1 [Tool Condition](#x.3.1)
  * 7.3.2 [Machining Finalized](#x.3.2)
  * 7.3.3 [Passed Visual Inspection](#x.3.3)
  

<br>

---

<a name='x.0'></a>

## 7.0 Preparing Environment and Importing Data

[back to top](#top)

<a name='x.0.1'></a>

### 7.0.1 Import Packages

[back to top](#top)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import interact, ToggleButtons, SelectMultiple
import seaborn as sns
from scipy import signal

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

<a name='x.0.1'></a>

### 7.0.2 Load Dataset

[back to top](#top)

In [2]:
status = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                     "ds_for_engineers/main/data/mill_tool_wear/train.csv")
status

,No,material,feedrate,clamp_pressure,tool_condition,machining_finalized,passed_visual_inspection
0,1,wax,6,4.0,unworn,yes,yes
1,2,wax,20,4.0,unworn,yes,yes
2,3,wax,6,3.0,unworn,yes,yes
3,4,wax,6,2.5,unworn,no,NaN
4,5,wax,20,3.0,unworn,no,NaN
5,6,wax,6,4.0,worn,yes,no
6,7,wax,20,4.0,worn,no,NaN
7,8,wax,20,4.0,worn,yes,no
8,9,wax,15,4.0,worn,yes,no
9,10,wax,12,4.0,worn,yes,no


In [3]:
tests = pd.DataFrame()

for i in range(1,19):
  if i < 10:
    df = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_0{}.csv".format(i))
  else:
    df = pd.read_csv("https://raw.githubusercontent.com/wesleybeckner/"\
                   "ds_for_engineers/main/data/mill_tool_wear/"\
                   "experiment_{}.csv".format(i))
  df["feedrate"] = status.loc[i-1,"feedrate"]
  df["clamp_pressure"] = status.loc[i-1,"clamp_pressure"]
  df["tool_condition"] = status.loc[i-1,"tool_condition"]
  df["experiment"] = i
  tests = pd.concat([tests, df], ignore_index=True)

<a name='x.1'></a>

## 7.1 Exploratory Data Analysis

[back to top](#top)

<a name='x.1.1'></a>

### 7.1.1 First Look: Shape, Nulls, Description

[back to top](#top)

In [4]:
tests.shape

(25286, 52)

In [5]:
tests.loc[tests.isnull().any(1)]

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process,feedrate,clamp_pressure,tool_condition,experiment


In [6]:
tests.dtypes

X1_ActualPosition            float64
X1_ActualVelocity            float64
X1_ActualAcceleration        float64
X1_CommandPosition           float64
X1_CommandVelocity           float64
X1_CommandAcceleration       float64
X1_CurrentFeedback           float64
X1_DCBusVoltage              float64
X1_OutputCurrent             float64
X1_OutputVoltage             float64
X1_OutputPower               float64
Y1_ActualPosition            float64
Y1_ActualVelocity            float64
Y1_ActualAcceleration        float64
Y1_CommandPosition           float64
Y1_CommandVelocity           float64
Y1_CommandAcceleration       float64
Y1_CurrentFeedback           float64
Y1_DCBusVoltage              float64
Y1_OutputCurrent             float64
Y1_OutputVoltage             float64
Y1_OutputPower               float64
Z1_ActualPosition            float64
Z1_ActualVelocity            float64
Z1_ActualAcceleration        float64
Z1_CommandPosition           float64
Z1_CommandVelocity           float64
Z

In [7]:
tests.describe()

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,X1_OutputPower,Y1_ActualPosition,Y1_ActualVelocity,Y1_ActualAcceleration,Y1_CommandPosition,Y1_CommandVelocity,Y1_CommandAcceleration,Y1_CurrentFeedback,Y1_DCBusVoltage,Y1_OutputCurrent,Y1_OutputVoltage,Y1_OutputPower,Z1_ActualPosition,Z1_ActualVelocity,Z1_ActualAcceleration,Z1_CommandPosition,Z1_CommandVelocity,Z1_CommandAcceleration,Z1_CurrentFeedback,Z1_DCBusVoltage,Z1_OutputCurrent,Z1_OutputVoltage,S1_ActualPosition,S1_ActualVelocity,S1_ActualAcceleration,S1_CommandPosition,S1_CommandVelocity,S1_CommandAcceleration,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,feedrate,clamp_pressure,experiment
count,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,2.528600e+04,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,2.528600e+04,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.0,25286.0,25286.0,25286.0,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,2.528600e+04,25286.000000,2.528600e+04,25286.000000,25286.000000,25286.000000,25286.0,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000,25286.000000
mean,159.052045,-0.288657,0.094264,159.050700,-0.283076,0.253215,-0.469714,6.603073e-02,326.945859,7.986942,0.000610,99.230064,-0.422932,0.928832,99.226271,-0.412075,1.484158,-0.061681,6.398315e-02,325.862058,7.068352,0.000637,47.780638,-0.328759,-0.103658,47.778031,-0.328464,0.134118,0.0,0.0,0.0,0.0,-115.373082,38.986424,0.248602,-115.051536,38.650012,3.855889e-01,15.243948,6.692938e-01,322.784505,85.479195,0.127405,12.0,1.003441,47.345013,16.542039,7.123942,3.368168,10.738235
std,19.330873,5.658260,93.877623,19.331144,5.664309,72.594951,4.220750,3.700384e-02,1.459937,7.710379,0.001565,29.244880,6.006439,85.074580,29.242802,6.004721,81.358073,4.469548,4.777806e-02,1.804164,8.601484,0.002098,34.255650,7.635223,66.442671,34.252517,7.637045,61.988683,0.0,0.0,0.0,0.0,1212.730873,23.491267,32.204079,1212.766136,23.758090,6.189738e+00,10.222419,4.332413e-01,4.293571,52.531863,0.080753,0.0,0.349055,43.826214,19.620219,6.167036,0.615639,5.213285
min,141.000000,-20.400000,-1280.000000,141.000000,-20.000000,-1000.000000,-23.400000,2.780000e-19,320.000000,0.000000,-0.006060,72.400000,-32.800000,-1260.000000,72.400000,-32.400000,-1000.000000,-27.800000,2.680000e-19,319.000000,0.000000,-0.004920,27.500000,-51.500000,-1260.000000,27.500000,-50.000000,-1000.000000,0.0,0.0,0.0,0.0,-2150.000000,-0.069000,-150.000000,-2150.000000,0.000000,-9.540000e-07,-8.280000,0.000000e+00,290.000000,0.000000,-0.002960,12.0,0.000000,0.000000,3.000000,3.000000,2.500000,1.000000
25%,145.000000,-2.050000,-31.300000,145.000000,-2.050000,0.000000,-3.930000,4.150000e-02,326.000000,2.590000,0.000000,77.500000,-0.075000,-18.800000,77.500000,0.000000,0.000000,-3.090000,2.190000e-02,325.000000,1.810000,0.000000,28.500000,0.000000,-6.250000,28.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,-1160.000000,0.002000,-15.900000,-1160.000000,0.000000,0.000000e+00,0.821000,2.790000e-19,320.000000,0.000000,0.000005,12.0,1.000000,2.000000,3.000000,3.000000,3.000000,6.000000
50%,153.000000,0.000000,0.000000,153.000000,0.000000,0.000000,-0.666000,6.680000e-02,327.000000,7.140000,0.000174,90.000000,0.000000,0.000000,90.000000,0.000000,0.000000,0.146000,5.780000e-02,326.000000,4.930000,0.000004,29.500000,0.000000,0.000000,29.500000,0.000000,0.000000,0.0,0.0,0.0,0.0,-112.500000,53.300000,0.000000,-112.000000,53.300000,0.000000e+00,18.800000,8.580000e-01,322.000000,117.000000,0.164000,12.0,1.000000,39.000000,6.000000,3.000000,3.000000,12.000000
75%,162.000000,0.200000,25.000

<a name='x.1.2'></a>

### 7.1.2 Descriptive Statistics

[back to top](#top)

<a name='x.1.2.1'></a>

#### 7.1.2.1 Univariate Analysis

[back to top](#top)

In [8]:
tests.skew()

X1_ActualPosition             1.219721
X1_ActualVelocity             1.727296
X1_ActualAcceleration         0.912499
X1_CommandPosition            1.219765
X1_CommandVelocity            1.715114
X1_CommandAcceleration       -0.027659
X1_CurrentFeedback            0.269644
X1_DCBusVoltage               0.830362
X1_OutputCurrent             -0.037263
X1_OutputVoltage              2.844746
X1_OutputPower                7.594701
Y1_ActualPosition             1.199451
Y1_ActualVelocity            -1.448097
Y1_ActualAcceleration         0.841916
Y1_CommandPosition            1.199722
Y1_CommandVelocity           -1.446608
Y1_CommandAcceleration        1.953117
Y1_CurrentFeedback           -0.058733
Y1_DCBusVoltage               0.963220
Y1_OutputCurrent              1.084588
Y1_OutputVoltage              3.334789
Y1_OutputPower                6.154120
Z1_ActualPosition             1.477857
Z1_ActualVelocity             0.806090
Z1_ActualAcceleration        -1.073989
Z1_CommandPosition       

In [9]:
tests.kurtosis()

X1_ActualPosition              0.077315
X1_ActualVelocity             21.078788
X1_ActualAcceleration         69.964697
X1_CommandPosition             0.077357
X1_CommandVelocity            20.806996
X1_CommandAcceleration       143.523032
X1_CurrentFeedback             0.218157
X1_DCBusVoltage                4.465559
X1_OutputCurrent               0.873106
X1_OutputVoltage              14.430564
X1_OutputPower                88.354746
Y1_ActualPosition              0.032372
Y1_ActualVelocity             22.919821
Y1_ActualAcceleration         84.394353
Y1_CommandPosition             0.033187
Y1_CommandVelocity            23.067341
Y1_CommandAcceleration       111.688216
Y1_CurrentFeedback             0.886280
Y1_DCBusVoltage                1.698791
Y1_OutputCurrent               2.819351
Y1_OutputVoltage              14.875838
Y1_OutputPower                48.488226
Z1_ActualPosition              0.380075
Z1_ActualVelocity             30.490088
Z1_ActualAcceleration        220.670604


<a name='x.1.2.2'></a>

#### 7.1.2.2 Multivariate Analysis

[back to top](#top)

<a name='x.1.2.3'></a>

#### 7.1.2.3 Frequency Analysis

[back to top](#top)

In [10]:
dropdown = []
for feature in tests.columns:
  df = tests.loc[tests['experiment'] == 1]
  t = df.index
  s = df[feature]
  try:
    F = np.fft.fft(s)
    freq = np.fft.fftfreq(t.shape[-1])
    magnitude = np.sqrt(F.real**2 + F.imag**2)
    skew = pd.Series(magnitude).skew()
    kurt = pd.Series(magnitude).kurtosis()
    dropdown.append([feature, skew, kurt])
  except:
    print(feature)
    pass

Machining_Process
tool_condition


In [11]:
pd.DataFrame(dropdown).sort_values(2)

,0,1,2
24,Z1_ActualAcceleration,0.569778,-0.470645
13,Y1_ActualAcceleration,0.425063,-0.427739
37,S1_CommandAcceleration,1.116458,-0.193898
27,Z1_CommandAcceleration,0.518229,-0.113210
5,X1_CommandAcceleration,0.419607,-0.044612
2,X1_ActualAcceleration,0.525094,-0.018090
31,Z1_OutputVoltage,0.000000,0.000000
30,Z1_OutputCurrent,0.000000,0.000000
29,Z1_DCBusVoltage,0.000000,0.000000
28,Z1_CurrentFeedback,0.000000,0.000000


In [12]:
# first 5 are not useful
features = pd.DataFrame(dropdown).sort_values(2, ascending=False)\
  .reset_index(drop=True)[0].values[5:]

In [13]:
def fft(exp=np.arange(1,19), feature=features):
  df = tests.loc[tests['experiment'] == exp]
  t = df.index
  s = df[feature]
  F = np.fft.fft(s)
  freq = np.fft.fftfreq(t.shape[-1])
  fig, ax = plt.subplots(1,2,figsize=(10,5))
  ax[0].plot(t,s)
  ax[0].set_title("{}".format(df['tool_condition'][0]))
  magnitude = np.sqrt(F.real**2 + F.imag**2)
  skew = pd.Series(magnitude).skew()
  kurt = pd.Series(magnitude).kurtosis()
  ax[1].plot(freq, magnitude)
  ax[1].set_title("skew: {:.2f}, kurtosis: {:.2f}".format(skew, kurt))
  magnitude

In [14]:
interact(fft)

interactive(children=(Dropdown(description='exp', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, …

<function __main__.fft>

<a name='x.2'></a>

## 7.2 Feature Engineering

[back to top](#top)

<a name='x.2.3'></a>

### 7.2.3 Feature Normalization

[back to top](#top)

If we were using a linear model, or interested in comparing features based on coefficients we would want to do some feature scaling

<a name='x.2.4'></a>

### 7.2.4 Feature Selection

[back to top](#top)

An interesting improvement could be a lag or window based treatment of the time series data.

<a name='x.2.5'></a>

### 7.2.5 Fast Fourier Transform

[back to top](#top)

Here I've created some summary features for each experiment, proved not very useful.

In [18]:
fft = []
dicc = {}
for exp in range(1,19):
  dic = {}
  for feature in tests.columns:
    df = tests.loc[tests['experiment'] == exp]
    t = df.index
    s = df[feature]
    try:
      F = np.fft.fft(s)
      freq = np.fft.fftfreq(t.shape[-1])
      magnitude = np.sqrt(F.real**2 + F.imag**2)
      skew = pd.Series(magnitude).skew()
      kurt = pd.Series(magnitude).kurtosis()
      dropdown.append([feature, skew, kurt])
      high_amp = np.max(magnitude)
      high_freq = freq[np.argmax(np.max(magnitude))]  
      fft.append([feature,skew,kurt,high_amp])
      dic[f'{feature}_skew'] = skew
      dic[f'{feature}_kurt'] = kurt
      dic[f'{feature}_max'] = high_amp
    except:
      pass
  dicc[exp-1] = dic

In [19]:
pd.DataFrame(dicc).T.join(status['tool_condition'])

,X1_ActualPosition_skew,X1_ActualPosition_kurt,X1_ActualPosition_max,X1_ActualVelocity_skew,X1_ActualVelocity_kurt,X1_ActualVelocity_max,X1_ActualAcceleration_skew,X1_ActualAcceleration_kurt,X1_ActualAcceleration_max,X1_CommandPosition_skew,X1_CommandPosition_kurt,X1_CommandPosition_max,X1_CommandVelocity_skew,X1_CommandVelocity_kurt,X1_CommandVelocity_max,X1_CommandAcceleration_skew,X1_CommandAcceleration_kurt,X1_CommandAcceleration_max,X1_CurrentFeedback_skew,X1_CurrentFeedback_kurt,X1_CurrentFeedback_max,X1_DCBusVoltage_skew,X1_DCBusVoltage_kurt,X1_DCBusVoltage_max,X1_OutputCurrent_skew,X1_OutputCurrent_kurt,X1_OutputCurrent_max,X1_OutputVoltage_skew,X1_OutputVoltage_kurt,X1_OutputVoltage_max,X1_OutputPower_skew,X1_OutputPower_kurt,X1_OutputPower_max,Y1_ActualPosition_skew,Y1_ActualPosition_kurt,Y1_ActualPosition_max,Y1_ActualVelocity_skew,Y1_ActualVelocity_kurt,Y1_ActualVelocity_max,Y1_ActualAcceleration_skew,...,S1_CommandAcceleration_skew,S1_CommandAcceleration_kurt,S1_CommandAcceleration_max,S1_CurrentFeedback_skew,S1_CurrentFeedback_kurt,S1_CurrentFeedback_max,S1_DCBusVoltage_skew,S1_DCBusVoltage_kurt,S1_DCBusVoltage_max,S1_OutputCurrent_skew,S1_OutputCurrent_kurt,S1_OutputCurrent_max,S1_OutputVoltage_skew,S1_OutputVoltage_kurt,S1_OutputVoltage_max,S1_OutputPower_skew,S1_OutputPower_kurt,S1_OutputPower_max,S1_SystemInertia_skew,S1_SystemInertia_kurt,S1_SystemInertia_max,M1_CURRENT_PROGRAM_NUMBER_skew,M1_CURRENT_PROGRAM_NUMBER_kurt,M1_CURRENT_PROGRAM_NUMBER_max,M1_sequence_number_skew,M1_sequence_number_kurt,M1_sequence_number_max,M1_CURRENT_FEEDRATE_skew,M1_CURRENT_FEEDRATE_kurt,M1_CURRENT_FEEDRATE_max,feedrate_skew,feedrate_kurt,feedrate_max,clamp_pressure_skew,clamp_pressure_kurt,clamp_pressure_max,experiment_skew,experiment_kurt,experiment_max,tool_condition
0,32.358518,1049.654223,160358.0,7.241241,78.608798,2267.403657,0.525094,-0.018090,8459.839213,32.358710,1049.662474,160359.0,7.110567,76.197827,2267.005980,0.419607,-0.044612,6331.522668,10.235550,129.194311,1995.362517,29.839524,940.076420,86.095100,32.480693,1054.996955,343799.0,25.071505,736.581021,11347.3820,14.088407,321.947449,0.891809,31.551351,1013.552642,92779.9,4.808600,29.366344,1748.729717,0.425063,...,1.116458,-0.193898,499.999981,31.104582,995.349836,21636.816590,31.216578,1000.145261,951.305700,32.479049,1054.925646,337797.0,31.243408,1000.903649,120785.080,31.256235,1001.668769,181.344193,32.480764,1055.0,12660.0,32.480764,1055.000000,1055.0,23.399266,646.973129,67672.0,11.986209,227.320260,9146.0,32.480764,1055.0,6330.0,32.480764,1055.0,4220.0,32.480764,1055.0,1055.0,unworn
1,40.295421,1638.194077,308372.0,2.626479,8.069969,1277.580811,0.562289,0.293123,13088.655455,40.293702,1638.099747,308391.0,2.654007,8.140925,1305.110204,0.599825,0.158215,11321.882159,2.558626,8.482396,635.715798,9.304873,187.732198,42.956200,40.840674,1667.973734,545153.0,7.483621,126.746626,7919.0410,5.813683,84.460021,1.124780,38.686557,1550.555694,230642.1,2.461187,7.211775,1262.065195,0.588225,...,1.115913,-0.195628,499.999990,6.136065,77.727803,6692.531630,7.865156,129.686369,336.875400,40.835828,1667.709610,544911.0,7.986397,133.040693,41963.820,8.070305,135.662615,63.424294,40.841156,1668.0,20016.0,23.099240,770.143865,1755.0,6.328647,85.642701,24093.0,37.206297,1470.640823,71406.0,40.841156,1668.0,33360.0,40.841156,1668.0,6672.0,40.841156,1668.0,3336.0,unworn
2,38.342509,1486.687728,244091.0,2.627967,8.000897,918.946257,0.746675,0.622838,10160.781483,38.342372,1486.680555,244088.0,2.578902,7.689968,926.934184,0.655832,0.350907,8184.940463,2.786419,9.814904,940.782235,32.003336,1161.770970,101.481800,38.999256,1520.961300,497611.0,25.169811,831.559094,12362.5752,10.040535,221.253074,0.908206,35.462183,1337.016074,154388.3,2.649535,7.248651,976.465955,0.668690,...,1.382662,0.509055,599.999963,25.890431,862.017634,21802.929850,25.356344,836.422543,948.540000,38.992268,1520.597414,491131.0,25.407972,838.276907,120622.700,25.657187,850.311335,180.472302,39.000000,1521.

<a name='x.3'></a>

## 7.3 Modeling

[back to top](#top)

<a name='x.3.1'></a>

### 7.3.1 Tool Condition

[back to top](#top)

In [17]:
X = tests.drop(columns=['tool_condition', 'Machining_Process', 'experiment'],axis=1)
y = np.array(tests['tool_condition'])
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8,
                                                    random_state=42)

#### 7.3.1.1 Random Forest

In [ ]:
model = RandomForestClassifier(n_estimators=500, min_samples_leaf=4)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = model.predict(X_test)
print(f'{accuracy_score(y_pred, y_test):.4f}')
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

0.9949


array([[2357,   21],
       [   5, 2675]])

In [ ]:
# grab feature importances
imp = model.feature_importances_

# their std
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

# create new dataframe
feat = pd.DataFrame([X.columns, imp, std]).T
feat.columns = ['name', 'importance', 'std']
feat = feat.sort_values('importance', ascending=False)
feat = feat.reset_index(drop=True)
feat.columns = ['feature', 'importance', 'std']

In [ ]:
px.bar(feat, x='feature', y='importance', error_y='std')

#### 7.3.1.2 Gradient Boosting

In [ ]:
model = GradientBoostingClassifier(n_estimators=500, min_samples_leaf=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'{accuracy_score(y_pred, y_test):.4f}')
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

0.9931


array([[2350,   28],
       [   7, 2673]])

In [ ]:
# grab feature importances
imp = model.feature_importances_

# their std
# std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

# create new dataframe
feat = pd.DataFrame([X.columns, imp]).T
feat.columns = ['name', 'importance']
feat = feat.sort_values('importance', ascending=False)
feat = feat.reset_index(drop=True)
feat.columns = ['feature', 'importance']

In [ ]:
px.bar(feat, x='feature', y='importance')

#### 7.3.1.3 Extreme Gradient Boosting

In [ ]:
model = XGBClassifier(n_estimators=500, min_samples_leaf=4)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'{accuracy_score(y_pred, y_test):.4f}')
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

0.9955


array([[2361,   17],
       [   6, 2674]])

In [ ]:
# grab feature importances
imp = model.feature_importances_

# their std
# std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)

# create new dataframe
feat = pd.DataFrame([X.columns, imp]).T
feat.columns = ['name', 'importance']
feat = feat.sort_values('importance', ascending=False)
feat = feat.reset_index(drop=True)
feat.columns = ['feature', 'importance']

In [ ]:
px.bar(feat, x='feature', y='importance')

<a name='x.3.2'></a>

### 7.3.2 Machining Finalized

[back to top](#top)

<a name='x.3.3'></a>

### 7.3.3 Passed Visual Inspection

[back to top](#top)